In [6]:
import os
import pandas as pd

# 设置文件夹路径和CSV文件路径
folder_path = '../archive/masked_4K_ori/'
csv_file_path = '../archive/boneage-training-dataset.csv'
output_csv_path = '../archive/masked_4K.csv'

# 读取CSV文件和列出文件夹下的图片
image_files = os.listdir(folder_path)
image_ids = [int(os.path.splitext(file)[0]) for file in image_files]

# 读取CSV文件
df = pd.read_csv(csv_file_path)

# print(df['id'])

# 筛选出符合文件夹下图片ID的行
filtered_df = df[df['id'].isin(image_ids)]

# 将结果保存到新的CSV文件中
filtered_df.to_csv(output_csv_path, index=False)


In [1]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# 设置文件夹路径和CSV文件路径
folder_path = '../archive/masked_4K_ori/'
csv_file_path = '../archive/masked_4K.csv'
output_folder_base = '../archive/masked_4K_fold/'

# 读取CSV文件
df = pd.read_csv(csv_file_path)

# 获取图片ID和对应的标签
image_ids = df['id'].values
labels = df['boneage'].values  # 替换为实际的标签列名

# 设置K折交叉验证的折数
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=1)

# 创建输出文件夹
for i, (train_index, valid_index) in enumerate(skf.split(image_ids, labels), 1):
    output_folder = os.path.join(output_folder_base, f'fold_{i}')
    
    # 创建训练集和验证集文件夹
    train_folder = os.path.join(output_folder, 'train')
    valid_folder = os.path.join(output_folder, 'valid')
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(valid_folder, exist_ok=True)

    # 复制图片到对应的文件夹
    train_image_ids = image_ids[train_index]
    valid_image_ids = image_ids[valid_index]

    for image_id in train_image_ids:
        src_path = os.path.join(folder_path, f'{image_id}.png')  # 假设图片文件名是"{image_id}.jpg"
        dst_path = os.path.join(train_folder, f'{image_id}.png')
        shutil.copy(src_path, dst_path)

    for image_id in valid_image_ids:
        src_path = os.path.join(folder_path, f'{image_id}.png')
        dst_path = os.path.join(valid_folder, f'{image_id}.png')
        shutil.copy(src_path, dst_path)

    # 创建训练集和验证集的CSV文件
    train_csv = df[df['id'].isin(train_image_ids)]
    valid_csv = df[df['id'].isin(valid_image_ids)]

    train_csv.to_csv(os.path.join(output_folder, 'train.csv'), index=False)
    valid_csv.to_csv(os.path.join(output_folder, 'valid.csv'), index=False)


d:\Anaconda\envs\BAA\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
